# Model Gateway Tests 

The model gateway provides a kafka to http bridge.

In [ ]:
!pip install kafka-python

In [ ]:
from kafka import KafkaProducer
def produceIrisJson():
    value='{"inputs": [{"name": "predict", "shape": [1, 4], "datatype": "FP32", "data": [[1, 2, 3, 4]]}]}'
    b=str.encode(value)
    producer = KafkaProducer(bootstrap_servers=BROKER_IP+':9092')
    producer.send("iris-in",b)
    producer.flush()
    producer.close()

In [ ]:
from kafka import KafkaConsumer

def consumeIrisJson():
    consumer = KafkaConsumer(
         bootstrap_servers=[BROKER_IP+':9092'],
         group_id='my-consumer-1',
    )
    consumer.subscribe(['iris-out'])
    
    try:
        for msg in consumer:
            print(msg.value)
            break
    finally:
        consumer.close()


In [ ]:
import v2_dataplane_pb2 as v2
from google.protobuf.json_format import Parse

def produceIrisGrpc():
    data = '{"model_name":"iris","inputs":[{"name":"input","contents":{"fp32_contents":[1,2,3,4]},"datatype":"FP32","shape":[1,4]}]}'
    msg = Parse(data,v2.ModelInferRequest())
    b = msg.SerializeToString()
    producer = KafkaProducer(bootstrap_servers=BROKER_IP+':9092')
    producer.send("iris-in",b)
    producer.flush()
    producer.close()

In [ ]:
from kafka import KafkaConsumer

def consumeIrisGrpc():
    consumer = KafkaConsumer(
         bootstrap_servers=[BROKER_IP+':9092'],
         group_id='my-consumer-1',
    )
    consumer.subscribe(['iris-out'])
    
    try:
        for msg in consumer:
            resp = v2.ModelInferResponse()
            resp.ParseFromString(msg.value)
            print(resp)
            break
   
    finally:
        consumer.close()

# Model Gateway Local Test


## Setup


* `make start-all-mlserver`
* Update the config file `config/agent.yaml` locally 
```
kafka:
  active: true
  broker: "kafka:29092"
```

Note: Seems not to presently work with confluent-kafka with Docker compose kraft (no zookeeper) Kafka.



In [ ]:
BROKER_IP="0.0.0.0"

In [ ]:
!grpcurl -d '{"model":{ \
              "meta":{"name":"iris"},\
              "modelSpec":{"uri":"gs://seldon-models/mlserver/iris",\
                           "requirements":["sklearn"],\
                           "memoryBytes":500},\
              "deploymentSpec":{"replicas":1},\
              "streamSpec":{"inputTopic":"iris-in","outputTopic":"iris-out"}}}' \
         -plaintext \
         -import-path ../../apis \
         -proto ../../apis/mlops/scheduler/scheduler.proto  0.0.0.0:9004 seldon.mlops.scheduler.Scheduler/LoadModel

In [ ]:
!grpcurl -d '{"name":"mlserver"}' \
         -plaintext \
         -proto ../apis/mlops/scheduler/scheduler.proto  0.0.0.0:9004 seldon.mlops.scheduler.Scheduler/ServerStatus

In [ ]:
!curl -v http://0.0.0.0:9000/v2/models/iris/infer -H "Content-Type: application/json" -H "seldon-model: iris"\
        -d '{"inputs": [{"name": "predict", "shape": [1, 4], "datatype": "FP32", "data": [[1, 2, 3, 4]]}]}'

In [ ]:
produceIrisJson()
consumeIrisJson()

In [ ]:
produceIrisGrpc()
consumeIrisGrpc()

In [ ]:
!grpcurl -d '{"model":{"name":"iris"}}' \
         -plaintext \
         -import-path ../../apis \
         -proto ../../apis/mlops/scheduler/scheduler.proto  0.0.0.0:9004 seldon.mlops.scheduler.Scheduler/UnloadModel

# Model Gateway K8S Test


## Setup


* Install on k8s
* Update the config map seldon-agent 
```
kafka:
  active: true
  broker: "seldon-kafka-plain-bootstrap.kafka:9092"
```

In [ ]:
BROKER_IP=!kubectl get svc seldon-kafka-plain-bootstrap -n kafka -o jsonpath='{.status.loadBalancer.ingress[0].ip}'
BROKER_IP=BROKER_IP[0]
BROKER_IP

In [ ]:
SCHEDULER_IP=!kubectl get svc seldon-scheduler -n seldon-mesh -o jsonpath='{.status.loadBalancer.ingress[0].ip}'
SCHEDULER_IP=SCHEDULER_IP[0]
import os
os.environ['SCHEDULER_IP'] = SCHEDULER_IP
SCHEDULER_IP

In [ ]:
MESH_IP=!kubectl get svc seldon-mesh -n seldon-mesh -o jsonpath='{.status.loadBalancer.ingress[0].ip}'
MESH_IP=MESH_IP[0]
import os
os.environ['MESH_IP'] = MESH_IP
MESH_IP

In [ ]:
!grpcurl -d '{"model":{ \
              "meta":{"name":"iris"},\
              "modelSpec":{"uri":"gs://seldon-models/mlserver/iris",\
                           "requirements":["sklearn"],\
                           "memoryBytes":500},\
              "deploymentSpec":{"replicas":1},\
              "streamSpec":{"inputTopic":"iris-in","outputTopic":"iris-out"}}}' \
         -plaintext \
         -import-path ../../apis \
         -proto ../../apis/mlops/scheduler/scheduler.proto  ${SCHEDULER_IP}:9004 seldon.mlops.scheduler.Scheduler/LoadModel

In [ ]:
!grpcurl -d '{"name":"mlserver"}' \
         -plaintext \
         -proto ../apis/mlops/scheduler/scheduler.proto  ${SCHEDULER_IP}:9004 seldon.mlops.scheduler.Scheduler/ServerStatus

In [ ]:
!curl -v http://${MESH_IP}:80/v2/models/iris/infer -H "Content-Type: application/json" -H "seldon-model: iris"\
        -d '{"inputs": [{"name": "predict", "shape": [1, 4], "datatype": "FP32", "data": [[1, 2, 3, 4]]}]}'

In [ ]:
produceIrisJson()
consumeIrisJson()

In [ ]:
produceIrisGrpc()
consumeIrisGrpc()

In [ ]:
!grpcurl -d '{"model":{"name":"iris"}}' \
         -plaintext \
         -import-path ../../apis \
         -proto ../../apis/mlops/scheduler/scheduler.proto  ${SCHEDULER_IP}:9004 seldon.mlops.scheduler.Scheduler/UnloadModel